In [96]:
from google.cloud import bigquery

import pandas as pd
import numpy as np
from datetime import date,datetime,timedelta,timezone

import math
import os

import tensorflow as tf
import tensorflow_decision_forests as tfdf  # constantly registered to load model 
print(tf.__version__)
print(tfdf.__version__)

2.12.0
1.4.0


In [97]:
# import functions_framework
# @functions_framework.http
# def xgb_tf_predict_incident_severity(request):

# Load Configuration Data and Constant Variable

In [98]:
projectId="pongthorn"
#_model='binary_xgb_tf_model'   # the is resonable because unsseen and tune 70%
all_prediction=False
is_evaluation=True

_model="tuned_binary_xgb_tf_model"  # best 75% on unseen test
model_version=f'{_model}_v1_t150723'

model_gs_path=f"gs://demo-tf-incident-pongthorn/{_model}"

dataset_id="SMartML"

data_table="new2_incident"
prediction_table="new2_result_binary_prediction_incident"


# unusedCols_unseen=['id','severity_name','imported_at','open_to_close_hour']

unusedCols_unseen=['id','severity_name','imported_at','range_open_to_close_hour']


# Get today's date
prediction_datetime=datetime.now(timezone.utc)
today_str=prediction_datetime.strftime("%Y-%m-%d")
today=datetime.strptime(today_str,"%Y-%m-%d")
print(f"Prediction at {prediction_datetime} for {today_str} ({today})")

print(model_gs_path)
print(f"Data: {data_table} and Prediction: {prediction_table}")

Prediction at 2023-07-21 17:21:58.941017+00:00 for 2023-07-21 (2023-07-21 00:00:00)
gs://demo-tf-incident-pongthorn/tuned_binary_xgb_tf_model
Data: new2_incident and Prediction: new2_result_binary_prediction_incident


# Load Model

In [99]:
abc_model = tf.keras.models.load_model(model_gs_path)  
print(abc_model.summary())
# abc_model = tf.keras.models.load_model(model_local_path) 

[INFO 23-07-21 17:22:01.4874 UTC kernel.cc:1243] Loading model from path gs://demo-tf-incident-pongthorn/tuned_binary_xgb_tf_model/assets/ with prefix fe39250a4ca44dae
[INFO 23-07-21 17:22:02.1517 UTC decision_forest.cc:660] Model loaded with 60 root(s), 4704 node(s), and 6 input feature(s).
[INFO 23-07-21 17:22:02.1527 UTC abstract_model.cc:1311] Engine "GradientBoostedTreesGeneric" built
[INFO 23-07-21 17:22:02.1533 UTC kernel.cc:1075] Use fast generic engine


Model: "gradient_boosted_trees_model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
Total params: 1
Trainable params: 0
Non-trainable params: 1
_________________________________________________________________
None


# BigQuery Configuration

In [100]:
client = bigquery.Client(project=projectId)
new_data_table_id=f"{projectId}.{dataset_id}.{data_table}"
predictResult_table_id=f"{projectId}.{dataset_id}.{prediction_table}"
print(new_data_table_id)
print(predictResult_table_id)

pongthorn.SMartML.new2_incident
pongthorn.SMartML.new2_result_binary_prediction_incident


In [101]:
try:
    client.get_table(predictResult_table_id)  # Make an API request.
    print("Predict Result Table {} already exists.".format(predictResult_table_id))
    
except Exception as ex:
    schema = [
    bigquery.SchemaField("id", "INTEGER", mode="REQUIRED"),
    bigquery.SchemaField("prediction_item_date", "DATE", mode="REQUIRED"),    
    bigquery.SchemaField("label_binary_severity", "INTEGER", mode="REQUIRED"),
    bigquery.SchemaField("pred_binary_severity", "INTEGER", mode="REQUIRED"),       
    bigquery.SchemaField("prediction_datetime", "DATETIME", mode="REQUIRED") ,
    bigquery.SchemaField("model_version", "STRING", mode="REQUIRED")     
    ]

    table = bigquery.Table(predictResult_table_id,schema=schema)
    table.time_partitioning = bigquery.TimePartitioning(
    type_=bigquery.TimePartitioningType.DAY,field="prediction_item_date")
    
    table = client.create_table(table)  # Make an API request.
    
    print(
        "Created table {}.{}.{}".format(table.project, table.dataset_id, table.table_id)
    )

Predict Result Table pongthorn.SMartML.new2_result_binary_prediction_incident already exists.


# Load data to Make Prediction

In [102]:
if int(all_prediction)==0:
    sql=f"""
    SELECT *  FROM `{new_data_table_id}` 
     WHERE DATE(imported_at) = '{today_str}'
     order by imported_at
    """
else:
    sql=f"""
    SELECT *  FROM `{new_data_table_id}` 
     order by imported_at
    """

print(sql)


query_result=client.query(sql)
df=query_result.to_dataframe()
if df.empty==True:
  print("no data to make prediction")  
  # return "no data to make prediction"
print(df.info())


    SELECT *  FROM `pongthorn.SMartML.new2_incident` 
     WHERE DATE(imported_at) = '2023-07-21'
     order by imported_at
    
no data to make prediction
<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 11 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   id                        0 non-null      object
 1   severity_id               0 non-null      object
 2   severity_name             0 non-null      object
 3   sla                       0 non-null      object
 4   product_type              0 non-null      object
 5   brand                     0 non-null      object
 6   service_type              0 non-null      object
 7   incident_type             0 non-null      object
 8   open_to_close_hour        0 non-null      object
 9   range_open_to_close_hour  0 non-null      object
 10  imported_at               0 non-null      object
dtypes: object(11)
memory usage: 0.0+ bytes
None


In [81]:
unseen =df.drop(columns=unusedCols_unseen)
print(unseen.info())
unseen.tail(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49 entries, 0 to 48
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   severity_id         49 non-null     Int64  
 1   sla                 49 non-null     object 
 2   product_type        49 non-null     object 
 3   brand               49 non-null     object 
 4   service_type        49 non-null     object 
 5   incident_type       49 non-null     object 
 6   open_to_close_hour  49 non-null     float64
dtypes: Int64(1), float64(1), object(5)
memory usage: 2.9+ KB
None


,severity_id,sla,product_type,brand,service_type,incident_type,open_to_close_hour
39,4,24x7 4Hrs Response Time,Firewall,Palo Alto,Request,Software,622.283333
40,2,24x7 4Hrs Resolution Time,Firewall,Palo Alto,Incident,General Incident,0.683333
41,2,24x7 4Hrs Response Time,Server,DELL,Incident,Network Card Failure,17.400000
42,3,24x7 4Hrs Response Time,Software,Red Hat,Incident,General Incident,4.200000
43,3,24x7 4Hrs Response Time,Software,VMWare,Incident,General Incident,1.233333
44,4,24x7 4Hrs Resolution Time,Software,VMWare,Request,Upgrade Software,5.000000
45,4,24x7 4Hrs Resolution Time,Software,VMWare,Request,Upgrade Software,2.833333
46,4,24x7 4Hrs Resolution Time,Software,VMWare,Request,Upgrade Software,2.500000
47,4,24x7 6Hrs Resolution Time,Storage,HPE,Request,OS / Firmware,2.500000
48,4,24x7 6Hrs Resolution Time,Storage,HPE,Request,OS / Firmware,15.283333


In [82]:
unseen_ds= tfdf.keras.pd_dataframe_to_tf_dataset(unseen.drop(columns=['severity_id']))
print(unseen_ds)

<_PrefetchDataset element_spec={'sla': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'product_type': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'brand': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'service_type': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'incident_type': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'open_to_close_hour': TensorSpec(shape=(None,), dtype=tf.float64, name=None)}>


In [83]:
predResultList=abc_model.predict(unseen_ds)
predServerityIDList=[]
for predResult in predResultList:
    # prob = tf.nn.sigmoid(predResult) # no need to convert to Signmoid
    # print(prob)
    # pred_seveirty_id= "critical" if _class==1 else "normal"
    _class= 1 if predResult[0]>=0.5 else 0  
    predServerityIDList.append(_class) #0=normal , 1=critical
    print(f"{predResult} : {_class}")

dfPred=pd.DataFrame(data=predServerityIDList,columns=["pred_binary_severity"])   


1/1 [==============================] - 0s 71ms/step
[0.25636742] : 0
[0.17903766] : 0
[0.9165269] : 1
[0.9285503] : 1
[0.87518704] : 1
[0.69971377] : 1
[0.9147515] : 1
[0.92415375] : 1
[0.84933454] : 1
[0.44458163] : 0
[0.9435532] : 1
[0.8062814] : 1
[0.7003334] : 1
[0.9286715] : 1
[0.5661504] : 1
[0.66304684] : 1
[0.9765022] : 1
[0.951737] : 1
[0.44791013] : 0
[0.9255828] : 1
[0.42805284] : 0
[0.45431378] : 0
[0.20582551] : 0
[0.76085263] : 1
[0.5264448] : 1
[0.53275955] : 1
[0.67538774] : 1
[0.19062455] : 0
[0.1809187] : 0
[0.7941052] : 1
[0.9313895] : 1
[0.2712011] : 0
[0.05013854] : 0
[0.03604104] : 0
[0.08623972] : 0
[0.01716065] : 0
[0.01691318] : 0
[0.01121346] : 0
[0.00765095] : 0
[0.02547074] : 0
[0.9886457] : 1
[0.92415375] : 1
[0.3675326] : 0
[0.3275662] : 0
[0.02060693] : 0
[0.0102007] : 0
[0.00765095] : 0
[0.00982146] : 0
[0.19293757] : 0


2023-07-21 17:06:52.373743: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [49]
	 [[{{node Placeholder/_0}}]]


In [84]:
def map_4to2_serverity(severity_id):
    if severity_id==1 or severity_id==2:
       return 1
    else:
       return 0 
df['label_binary_severity'] =df['severity_id'].apply(map_4to2_serverity)

dfPred
df=pd.concat([df,dfPred],axis=1)
# df

# Evaluate model

In [85]:
if is_evaluation:
    from sklearn.metrics import confusion_matrix,classification_report
    className=list(set().union(list(df['pred_binary_severity'].unique()),list(df['label_binary_severity'].unique())))
    print(className)
    actualClass=[  f'actual-{x}' for x in  className]
    predictedlClass=[  f'pred-{x}' for x in className]
    y_true=list(df['label_binary_severity'])
    y_pred=list(df['pred_binary_severity'])
    cnf_matrix = confusion_matrix(y_true,y_pred)
    cnf_matrix

    # #index=actual , column=prediction
    cm_df = pd.DataFrame(cnf_matrix,
                         index = actualClass, 
                         columns = predictedlClass)
    print(cm_df)
    print(classification_report(y_true, y_pred, labels=className))

[0, 1]
          pred-0  pred-1
actual-0      22       7
actual-1       3      17
              precision    recall  f1-score   support

           0       0.88      0.76      0.81        29
           1       0.71      0.85      0.77        20

    accuracy                           0.80        49
   macro avg       0.79      0.80      0.79        49
weighted avg       0.81      0.80      0.80        49



# Transform data for Writing Prediction Result to BQ

In [86]:
df=df[['id','label_binary_severity','pred_binary_severity']]
df['prediction_item_date']=today
df['prediction_datetime']=datetime.now()
df['model_version']=model_version
# print(df)

# Load ata to BQ

In [87]:
def loadDataFrameToBQ():
    # WRITE_TRUNCATE , WRITE_APPEND
    try:
        if all_prediction==1: 
            job_config = bigquery.LoadJobConfig(write_disposition="WRITE_TRUNCATE")
        else:
            job_config = bigquery.LoadJobConfig(write_disposition="WRITE_APPEND")

        job = client.load_table_from_dataframe(
            df, predictResult_table_id, job_config=job_config
        )
        job.result()  # Wait for the job to complete.
        print("Total Prediction ML ", len(df), "Imported bigquery successfully")

    except BadRequest as e:
        print("Bigquery Error\n")
        for e in job.errors:
            print('ERROR: {}'.format(e['message']))

try:
    loadDataFrameToBQ()
except Exception as ex:
    raise ex

Total Prediction ML  49 Imported bigquery successfully


In [ ]:
# return 'All incidents has been predicted completely.'